<img src="data/images/lecture-notebook-header.png" />

# Clustering: K-Means

K-means clustering is a popular unsupervised machine learning algorithm used for clustering data points into groups or clusters. It aims to partition the data into K clusters, where K is a predefined number. It's a centroid-based method yielding partitional, exclusive and complete clusterings, i.e., each data point is assigned to exactly one cluster. It's only input parameter is the number of clusters K. In corner cases such as a very bad initialization of the centroids, one more clusters might be empty.

The algorithm works by iteratively assigning data points to the nearest cluster centroid and recalculating the centroids based on the newly formed clusters. The process continues until convergence, which is reached when the assignments no longer change significantly or a maximum number of iterations is reached.

Here's a step-by-step explanation of the K-means clustering algorithm:

* **Initialization:** Choose the number of clusters K and randomly initialize K centroids in the feature space.

* **Assignment:** Assign each data point to the nearest centroid based on a distance metric, commonly the Euclidean distance. Each data point is assigned to the cluster with the closest centroid.

* **Update:** Recalculate the centroids of each cluster by taking the mean of all the data points assigned to that cluster. This step moves the centroids closer to the center of their respective clusters.

* **Iteration:** Repeat steps 2 and 3 until convergence. Convergence occurs when the assignments no longer change significantly or a maximum number of iterations is reached.

* **Output:** The algorithm outputs K clusters, each represented by its centroid, and the assignments of data points to clusters.

K-means clustering is a simple and efficient algorithm, but it has some limitations. It can be sensitive to the initial placement of centroids and is also affected by outliers. Additionally, the algorithm assumes that the clusters are spherical and have similar sizes, which may not hold true in all cases. Despite these limitations, K-means clustering remains a widely used technique for exploratory data analysis, customer segmentation, and pattern recognition tasks.

**Side note:** This notebook includes different evaluation metrics to assess the quality of clusterings, which will be covered a bit later in the course.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Import Required Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from tqdm import tqdm

---

## Playing with Toy Data

[`sklearn.datasets`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets) provides a series of methods to randomly generate sample data. Try different methods and see how the results will change. The method `make_blobs()` allows you to specify the number of features. However, for the scatter plots to show correctly, you need to stick with `n_features=2`.

In [ ]:
X, y = make_blobs(n_samples=100, centers=5, n_features=2, cluster_std=0.85, random_state=11)
#X, y = make_moons(n_samples=250, noise=0.105, random_state=0)
#X, y = make_circles(n_samples=500, noise=0.06, factor=0.5, random_state=0)

We can plot the data to get a first idea how our data looks like. Of course, in practice this might not be (trivially) possible with data points of more than 3 dimensions. The following example, however, focuses on illustrating the characteristics of K-Means.

In [ ]:
plt.figure()
plt.scatter(X[:,0], X[:,1])
plt.show()

The method below plots the clustering, and we will use it throughout the rest of the notebook. The input of the methods is an instance of [`sklearn.cluster.KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) of calculating the clusters.

In [ ]:
def plot_clusters(centroids, labels, data):
    plt.figure()
    
    # Plot all the data points a color-code them w.r.t. to their cluster label/id
    plt.scatter(data[:, 0], data[:, 1], c=labels, s=50, cmap=plt.cm.tab10)
    
    # Plot the cluster centroids as fat plus signs
    for cluster_id, centroid in enumerate(centroids):
        plt.scatter(centroid[0], centroid[1], marker='+', color='k', s=250, lw=5)

    plt.tight_layout()
    plt.show()

### Effects of Different Values for k (Number of Cluster)

The number of clusters k has to be specified by the user. In general, it's not obvious how to set $k$. The following loop calculates the K-Means clusterings with values for $k$ from 2 to 20. Each run (i.e., value of $k$) will give us a different Sum of Squared Error (SSE), Silhouette Score, and Adjusted Rand Index (ARI).

In [ ]:
sse, silhouette, ari = [], [], []

for k in tqdm(range(2, 20+1)):

    # Run K-Means for the current number of clusters k
    kmeans = KMeans(n_clusters=k).fit(X)
    
    # "inertia_" holds the sum of squared distances of samples to their closest cluster center
    # Record the current (k, sse) pair
    sse.append((k, kmeans.inertia_))
    
    # silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters    
    silhouette.append((k, silhouette_score(X, kmeans.labels_)))
    
    ari.append((k, adjusted_rand_score(y, kmeans.labels_)))
    
# Convert to numpy array for convenience
sse = np.array(sse)
silhouette = np.array(silhouette)    
ari = np.array(ari)

Let's plot the results -- first for the SSE values.

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('SSE', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in sse], [s[1] for s in sse], marker='o', lw=2)
plt.tight_layout()
plt.show()

Of course, the exact plot will differ depending on the type of data (blob, moons, circles), on the parameters when creating the data (e.g., the level of noise), as well as on the execution of K-Means itself -- note the K-Means is by default K-Means++, so the cluster centroids are not chosen randomly.

However, there are some general observations

* For increasing values of k, the SSE will always go down, and the SSE will always be 0 if $k$ is larger than the number of data points. This just derives directly from the definition of SSE.

* For (reasonably) well-separated blobs, we can see a clear "elbow" which indicates the best value for $k$. Try increasing the noise, and you should see that the elbow will become less and less pronounced.

* For non-blob data, the SEE will of course still go down with increasing values of $k$, but a clear elbow is typically missing.

Now the plot for the silhouette scores:

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('Silhouette Score', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in silhouette], [s[1] for s in silhouette], marker='o', lw=2)
plt.tight_layout()
plt.show()

Again, for well-separated blobs, the silhouette score is highest for the expected values of $k$. Adding more noise to the blobs or using non-blob-like data will also result in much less obvious trends. Check it out for different data samples generated using different parameters.

Lastly, we can plot the ARI values.

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('Rand Index', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in ari], [s[1] for s in ari], marker='o', lw=2)
plt.tight_layout()
plt.show()

Again, for well-seaprated cluster, we can easily observe a value for $k$ yielding the highest Rand Index.

### Plot Clusters for Different Values of $k$

To get a better sense how K-Means behaves, run K-Means for different generated data (with different parameters) and different values for $k$ and plot the results.

In [ ]:
kmeans = KMeans(n_clusters=5).fit(X)  # Change n_clusters and run the code cell again to plot the new clusters

plot_clusters(kmeans.cluster_centers_, kmeans.labels_, X)

### K-Means with Random Initialization of Centroids

As mentioned above, [`sklearn.cluster.KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) uses the K-means++ approach to initialize the centroids. By default, K-Means runs 10 times and picks the best results (check the `n_init` parameter).

We can make the algorithm manually "stupid" by setting `init='random'` and `n_init=1`. Note that `random` in this implementation means a random selection of data points -- and not an arbitrary point between data points. As such, the will never be any empty cluster, but potentially clusters with only one data point (the one that got chosen as initial centroid)

With this setting, let's run K-Means 100 times and record the worst results in terms of the largest SSE. You can run this method and plot several times and see the different results.

In [ ]:
worst_sse, worst_kmeans = 0, None

for _ in tqdm(range(0, 100)):

    # Run K-Means (k=5 here assumes the default sample data with 5 blobs; change as needed)
    kmeans = KMeans(n_clusters=10, init='random', n_init=1).fit(X)
    
    # Keep track of the worst run of K-Means over the dataset (worst = largest SSE)
    if kmeans.inertia_ > worst_sse:
        worst_sse = kmeans.inertia_
        worst_kmeans = kmeans

...and we can of course plot the clusters again, this time for the clusters with to worst SSE value, i.e., the worst clustering.

In [ ]:
plot_clusters(worst_kmeans.cluster_centers_, worst_kmeans.labels_, X)

Clearly, K-Means can suffer from bad/unfortunate initialization of the centroids. That is why all practical implementations use a more sophisticated method to initialize the clusters, e.g., K-Means++. In principle, since the K-Means++ initialization is still random (just not uniformly random), K-Means++ might still result in a suboptimal clustering. Compared to a more naive initialization, however, the probability for this to happen is much, much lower.

---

## K-Means over a Real-World Dataset

The [Pima Indians Diabetes Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database) is a well-known dataset frequently used in machine learning and data mining research. It contains information about a group of Pima Indian women from Arizona, USA, and their potential risk of developing diabetes. The dataset is commonly used for classification tasks, aiming to predict whether a person has diabetes or not based on various features.

Here are the details of the Pima diabetes dataset:

* Number of instances: 768
* Number of attributes: 8
* Target variable: Outcome (0 for non-diabetic, 1 for diabetic)

The attributes or features in the dataset are:

* Pregnancies: Number of times pregnant.
* Glucose: Plasma glucose concentration after a 2-hour oral glucose tolerance test.
* Blood Pressure: Diastolic blood pressure (mm Hg).
* Skin Thickness: Triceps skinfold thickness (mm).
* Insulin: 2-Hour serum insulin (mu U/ml).
* BMI: Body mass index (weight in kg / (height in m)^2).
* Diabetes Pedigree Function: Diabetes pedigree function, which provides an estimation of the genetic influence.
* Age: Age in years.

This is a convenient dataset since all attributes for the clustering are numerical, and we can use the default Euclidean distance as a similarity measure. First, as usual, we read the dataset from the file into a `pandas` DataFrame.

In [ ]:
df = pd.read_csv('data/datasets/diabetes/diabetes.csv')

df.head()

**Important:** Remember, in practice, you should always do at least a basic EDA to check for missing values and (obvious) outliers! We skip this step to keep the notebook simple.

First, we need to create the input data `X` for the clustering and -- for this example -- the label data `y`.

In [ ]:
X = df.drop(columns=['Outcome']).to_numpy()
y = df['Outcome'].to_numpy()

Optionally, we can normalize the data, e.g., using standardization. In fact, it is highly recommended that we do so as the domains of different attributes differ by several orders of magnitude. You can check again with the EDA and Data Preprocessing notebook regarding the importance of normalizing input features.

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Estimate Parameter Settings

Similar to the toy data above, we can evaluate the effects of varying $k$ on the clusterings.

In [ ]:
sse, silhouette, ari = [], [], []

for k in tqdm(range(2, 20+1)):

    # Run K-Means for the current number of clusters k
    kmeans = KMeans(n_clusters=k).fit(X)
    
    # "inertia_" holds the sum of squared distances of samples to their closest cluster center
    # Record the current (k, sse) pair
    sse.append((k, kmeans.inertia_))
    
    # silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters    
    silhouette.append((k, silhouette_score(X, kmeans.labels_)))
    
    ari.append((k, adjusted_rand_score(y, kmeans.labels_)))
    
# Convert to numpy array for convenience
sse = np.array(sse)
silhouette = np.array(silhouette)    
ari = np.array(ari)

And again, we can plot and inspect the results, starting the the SEE values.

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('SSE', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in sse], [s[1] for s in sse], marker='o', lw=2)
plt.tight_layout()
plt.show()

Compared to our toy dataset with the 5 well-separated blobs, here cannot see any well defined "elbow" that would allow us to unambiguously find the best value for $k$.

So let's have a look at the Silhouette Scores.

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('Silhouette Score', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in silhouette], [s[1] for s in silhouette], marker='o', lw=2)
plt.tight_layout()
plt.show()

Here we can now identify a maximum Silhouette Score when `k-4`. However, it is not as clear-cut as for the toy data since for `k=2` the Silhouette score is almost as high.

Finally, the plot for the Adjusted Rand Index (ARI):

In [ ]:
plt.figure()
plt.xlabel('K', fontsize=16)
plt.ylabel('Rand Index', fontsize=18)
plt.tick_params(axis="x", labelsize=12)
plt.tick_params(axis="y", labelsize=12)
plt.plot([s[0] for s in ari], [s[1] for s in ari], marker='o', lw=2)
plt.tight_layout()
plt.show()

These results also show that the best values for $k$ is arguably 4. Let's take this value to compute our "best" K-Means clustering; see the code cell below.

In [ ]:
best_kmeans = KMeans(n_clusters=4).fit(X)

### Visualization

As our dataset now has more than 3 attributes, we cannot direct plot the clusters. However, we can use dimensionality reduction techniques to map the data points into a lower-dimensional space. In this example, we use PCA to map our data into 2d -- a detailed discussion about PCA is beyond the scope here and covered in a dedicated notebook.

Using `scikit-learn`, the code is very simple. First, we train PCA based on the input data `X`. Using this trained model, we can now transform out data from the 3d space to the 2d space. For the plots, note that we also have to transform the location of the centroids from 3d to 2d.

In [ ]:
# Train PCA with 2 features/components as output
pca = PCA(n_components=2).fit(X)

# Transform input data from 3d to 2d
X_pca = pca.transform(X)

# Transform centroids from 3d to 3d
cluster_centers = pca.transform(best_kmeans.cluster_centers_)

Since we now have 2d data, we can call our method `plot_clusters` as usual to visualize the clusters together with the centroids.

In [ ]:
plot_clusters(cluster_centers, best_kmeans.labels_, X_pca)

As you will have noticed, some of the clusters overlap -- that is, the colored clusters are not separated -- as you would expect and you have seen in the plots above. The difference is that we performed the clustering by running K-Means in the 3d space and then used PCA to map the data into the 2d space. The issue is that dimensionality reduction always results in some loss of information. Here, we lose the separation of clusters in the plot, but only in the plot!

---

## Summary

K-Means is a very simple but very popular clustering algorithm. It offers several advantages that contribute to its popularity in the field of unsupervised machine learning:

* **Simplicity:** K-means is relatively easy to understand and implement. Its straightforward nature makes it accessible to beginners and allows for quick prototyping and exploration of data clusters. The algorithm's simplicity also contributes to its computational efficiency, making it suitable for large datasets.

* **Scalability:** K-means is a scalable algorithm that can handle large amounts of data efficiently. Its time complexity is linear with respect to the number of data points, making it suitable for applications where computational efficiency is crucial.

* **Interpretability:** The algorithm produces easily interpretable results. Each cluster is represented by its centroid, which serves as a prototype for the data points assigned to that cluster. This allows for meaningful insights and interpretation of the clusters, aiding in exploratory data analysis and pattern recognition.

However, K-means clustering also has some limitations that should be considered:

* **Sensitivity to Initialization:** The algorithm's performance is highly sensitive to the initial placement of centroids. Different initializations can lead to different clustering results. In some cases, it may converge to suboptimal solutions or get stuck in local optima. To mitigate this issue, multiple initializations or more advanced initialization techniques can be used.

* **Assumption of Spherical Clusters:** K-means assumes that the clusters are spherical and have similar sizes. However, real-world data often contains clusters with irregular shapes and varying sizes. In such cases, K-means may not perform well and may produce suboptimal or distorted clusters.

* **Handling Outliers:** K-means is sensitive to outliers, which can significantly influence the position of the cluster centroids. Outliers can lead to skewed cluster assignments and distort the clustering results. Preprocessing techniques such as outlier detection or using alternative clustering algorithms that are more robust to outliers may be necessary.

* **Determining the Number of Clusters (K):** Choosing the optimal number of clusters (K) is a challenging task. The algorithm requires the number of clusters to be specified in advance, and selecting an inappropriate K can lead to incorrect or misleading results. Various techniques, such as the elbow method or silhouette analysis, can be employed to estimate the optimal value of K, but it remains a subjective decision.

In summary, K-means clustering is a simple and efficient algorithm with advantages such as simplicity, scalability, and interpretability. However, it has limitations concerning initialization sensitivity, assumption of spherical clusters, handling outliers, and determining the optimal number of clusters. Despite these limitations, K-means clustering remains a widely used technique in various domains, and researchers continue to develop modifications and alternative algorithms to address its drawbacks.